In [1]:
#!/usr/bin/env python
import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image
import numpy as np
import cv2
from geometry_msgs.msg import PoseStamped
import matplotlib.pyplot as plt

In [2]:
label_with_coordinates = {'5': (406, 46), '3': (406, 407), '8': (165, 407), '1': (167, 45)}


In [11]:
def extract_points(dictionary):
    points = []
    labels = []
    for key, value in dictionary.items():
        points.append(value)
        labels.append(key)
    return points , labels

# Example usage:
points , labels = extract_points(label_with_coordinates)
print(points)
print(labels)

[(406, 46), (406, 407), (165, 407), (167, 45)]
['5', '3', '8', '1']


In [ ]:

def main():
   
    rospy.init_node('depth_point_extraction', anonymous=True)
    rospy.Subscriber('/3d_image/image_raw_depth', Image, depth_image_callback)
    rospy.spin()

In [10]:

def depth_image_callback(msg):
    try:
        # Convert ROS Image message to OpenCV image
        bridge = CvBridge()
        depth_image = bridge.imgmsg_to_cv2(msg, desired_encoding="passthrough")
        
        # Convert the depth image to a Numpy array
        depth_array = np.array(depth_image, dtype=np.float32)
        
        # Extract depth at the specified point
        depth_at_point = depth_array[point_y_pixels, point_x_pixels]
        point_z = depth_at_point
        rospy.loginfo("Depth at point (%d, %d): %.2f meters", point_x_pixels, point_y_pixels, depth_at_point)
        print('point Z',point_z)

        obj_x_m ,obj_y_m,obj_z_m  = get_x_y(point_z)
        get_orientation_from_xyz(obj_x_m , obj_y_m , obj_z_m)
    except CvBridgeError as e:
        rospy.logerr("CvBridge Error: %s", e)

In [7]:

def get_x_y(depth):
    
    fx = 525.0  # focal length in pixels
    fy = 525.0
    cx = 320.0  # principal point in pixels
    cy = 240.0
    # centers would get from the bounding box
    object_x_meters = (point_x_pixels - cx)*depth / fx
    object_y_meters = (point_y_pixels - cy)*depth / fy
    print ('Center point x & y in meters' , (object_x_meters , object_y_meters))
    #cx cy fx fy from camera calibration
    return object_x_meters , object_y_meters , depth

In [8]:

def get_orientation_from_xyz(objX, objY, objZ):
    # Normalize the object's position vector
    norm = np.linalg.norm([objX, objY, objZ])
    objX /= norm
    objY /= norm
    objZ /= norm
    
    # Define camera frame axes
    camera_x = [1, 0, 0]
    camera_y = [0, 1, 0]
    camera_z = [0, 0, 1]

    # Calculate the dot products between the camera frame axes and the object's position vector
    dot_x = np.dot(camera_x, [objX, objY, objZ])
    dot_y = np.dot(camera_y, [objX, objY, objZ])
    dot_z = np.dot(camera_z, [objX, objY, objZ])

    # Construct the rotation matrix
    rotation_matrix = np.array([[dot_x, dot_y, dot_z],
                                 [0, 0, 0],
                                 [0, 0, 0]])

    # Calculate the quaternion from the rotation matrix
    qw = np.sqrt(1 + rotation_matrix[0, 0] + rotation_matrix[1, 1] + rotation_matrix[2, 2]) / 2
    qx = (rotation_matrix[2, 1] - rotation_matrix[1, 2]) / (4 * qw)
    qy = (rotation_matrix[0, 2] - rotation_matrix[2, 0]) / (4 * qw)
    qz = (rotation_matrix[1, 0] - rotation_matrix[0, 1]) / (4 * qw)

    pose = [qx , qy , qz , qw]
   
    print([qx , qy , qz , qw])